<a href="https://colab.research.google.com/github/Heavy02011/test/blob/master/PLN_DonkeyCarRacing_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU allocation
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [0]:
import tensorflow
print(tensorflow.__version__)

## Git Clone the donkey repository

In [0]:
!git clone https://github.com/autorope/donkeycar.git 
%cd /content/donkeycar

!git checkout master

## Install donkey car

In [0]:
!pip3 install -e .[pc]

## Create Project

In [0]:
!donkey createcar --path /content/mycar

## Prepare Data
### Copy the following code and run on pi


```bash
cd ~/mycar/data
tar -czf tub_xx_yyyy_mm_dd.tar.gz tub_xx_yyyy_mm_dd

```

This will create a tub_xx_yyyy_mm_dd.tar.gz file under ~/mycar/data

## Upload Data


### Copy the tub to your local pc

Run this on your local pc if you are using linux/mac
```
sftp pi@raspberry.local
cd ~/mycar/data
get tub_xx_yyyy_mm_dd.tar.gz
```

If you are on a windows, download sftp utility like [filezilla](https://filezilla-project.org/) or [putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)

### Define your tub name here

In [0]:
tub_name="tub_49_19_12_10"

### Upload the tub to Google Drive

First upload the tub_x_yyyy_mm_dd.tar.gz to Google Drive. We will then mount Google Drive from colab and copy the data from Drive directly. When you run the cell below, you will need to click the link and generate an authorization code to for colab to access your drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Suppose you upload the tub_xx_yyyy_mm_dd.tar.gz to Google Drive/mycar/tub_xx_yyyy_mm_dd.tar.gz, this is how you copy it from Google Drive to colab

In [0]:
%cd /content/mycar/data
!cp /content/drive/My\ Drive/mycar/{tub_name}.tar.gz .

And untar it to the right place

In [0]:
!tar -xzf {tub_name}.tar.gz

Check whether the data is there

In [0]:
!ls {tub_name}

## Train your model

In [0]:
!python /content/mycar/manage.py train --type linear --model /content/mycar/models/mypilot.h5

Check if the model is generated



In [0]:
!ls -alh /content/mycar/models

In [0]:
%cd /content/mycar/models

import glob
file = glob.glob("*.png")
  
from IPython.display import Image
Image(file[0])

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [0]:
from google.colab import files

files.download('./mypilot.h5')

Alternatively, you can copy the model back to Google Drive too

In [0]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models/mypilot.h5

### Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

```
sftp pi@raspberry.local
cd mycar/models
put mypilot.h5
```

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Bonus - Salient Object Visualization

In [0]:
# !pip install git+https://github.com/autorope/keras-vis.git
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis
  
  

In [0]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/mypilot.h5 --type linear --salient

In [0]:
from google.colab import files

%cd /content/mycar
!ls -ahl
files.download('tub_movie.mp4')